In [1]:
#import tdt
import matplotlib.pyplot as plt
import trompy as tp
import numpy as np
import pickle
import pandas as pd
%matplotlib inline

In [4]:
with open('C://Data//hislight_pickles//adlib_light.pickle', 'rb') as handle:
    adlib_light = pickle.load(handle)

In [5]:
adlib_light

{'HL202_1': {'mouse_ID': 'HL202',
  'region': 'PVH',
  'virus': 'hislight',
  'diet': 'chow',
  'total_licks': 0,
  'solution': 'ensure',
  'corrected': array([-54.44102563, -54.44048115, -54.43985392, ..., -47.86257289,
         -47.84225283, -47.82145411]),
  'fs': 1017.2526245117188,
  'licks': []},
 'HL203_1': {'mouse_ID': 'HL203',
  'region': 'PVH',
  'virus': 'hislight',
  'diet': 'chow',
  'total_licks': 281,
  'solution': 'ensure',
  'corrected': array([-16.21146813, -16.21178564, -16.21211004, ..., -11.6774029 ,
         -11.67557085, -11.67380977]),
  'fs': 1017.2526245117188,
  'licks': array([ 256.41140224,  257.78634752,  258.48332288,  258.8483584 ,
          259.08232192,  259.3292288 ,  259.58334464,  259.72031488,
          259.84729088,  259.9723008 ,  260.10943488,  260.22526976,
          260.35634176,  260.48741376,  260.63945728,  260.77544448,
          260.90143744,  261.0274304 ,  261.16325376,  261.2944896 ,
          261.43440896,  261.56531712,  261.69524224

In [9]:
def get_mice(data):
    mice = []
    for key, val in data.items():
        if val["mouse_ID"] not in mice:
            mice.append(val["mouse_ID"])
    return mice

In [15]:
def get_lick_data (data, mice):
    
    licks = [] #empty list where data from all mice will be stored
    burst_number = []
    burst_size = []
    burst_size_first_3 = []
    
    for mouse in mice:
        temp_total = []
        temp_lick = []
        temp_burstnum = []
        temp_burstsize = []
        temp_burstsize_first_3 = []
        
        for key, val in data.items():
            if val["mouse_ID"] == mouse:
                onset= val["licks"]
                lickdata = tp.lickCalc(onset, minburstlength=3)
                temp_total.append(lickdata["total"])
                temp_lick.append(lickdata["licks"])
                temp_burstnum.append(lickdata["bNum"])
                temp_burstsize.append(lickdata["bMean"])
                temp_burstsize_first_3.append(lickdata["bMean-first3"])

        licks.append(np.mean(temp_total))
        burst_number.append(np.mean(temp_burstnum))
        burst_size.append(np.mean(temp_burstsize))
        burst_size_first_3.append(np.mean(temp_burstsize_first_3))
        
    data_to_return = {"licks":licks, "burst number": burst_number, "burst size": burst_size, "burst size first 3": burst_size_first_3}
    return data_to_return

In [16]:
mice= get_mice(adlib_light)

test= get_lick_data(adlib_light, mice)

Could not calculate burst probability
Could not calculate burst probability
Could not calculate burst probability
Optimal fit parameters not found
